In [ ]:
pip install transformers

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer1 = AutoTokenizer.from_pretrained("suriya7/Gemma-2B-Finetuned-Python-Model")
model1 = AutoModelForCausalLM.from_pretrained("suriya7/Gemma-2B-Finetuned-Python-Model")

import torch

In [ ]:
query = input('enter a query:')
prompt_template = f"""
<start_of_turn>user based on given instruction create a solution\n\nhere are the instruction {query}
<end_of_turn>\n<start_of_turn>model
"""
prompt = prompt_template
encodeds = tokenizer1(prompt, return_tensors="pt", add_special_tokens=True).input_ids

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model1.to(device)
inputs = encodeds.to(device)


# Increase max_new_tokens if needed
generated_ids = model1.generate(inputs, max_new_tokens=1000, do_sample=False, pad_token_id=tokenizer1.eos_token_id)
ans = ''
for i in tokenizer1.decode(generated_ids[0], skip_special_tokens=True).split('<end_of_turn>')[:2]:
    ans += i

# Extract only the model's answer
model_answer = ans.split("model")[1].strip()
print(model_answer)

In [ ]:
!pip install -q -U bitsandbytes==0.42.0
!pip install -q -U peft==0.8.2
!pip install -q -U trl==0.7.10
!pip install -q -U accelerate==0.27.1
!pip install -q -U transformers==4.38.0

In [ ]:
pip install -q -U datasets==2.16.1

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model_id = "suriya7/Gemma-2B-Finetuned-Python-Model"


model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

In [7]:
from datasets import load_dataset

dataset = load_dataset("flytech/python-codes-25k")
dataset

DatasetDict({
    train: Dataset({
        features: ['output', 'instruction', 'input', 'text'],
        num_rows: 49626
    })
})

In [8]:
dataset["train"][:20]["instruction"]

['Help me set up my daily to-do list!',
 'Create a shopping list based on my inputs!',
 'Calculate how much time I spend on my phone per week!',
 'Help me split the bill among my friends!',
 'Organize my movie list into genres!',
 'Calculate the average rating of my book collection!',
 'Create a playlist based on my mood!',
 'Help me find the best deals on my shopping list!',
 'Calculate how much I need to save per month for my vacation!',
 'Determine the most efficient route for my errands!',
 'Help me manage my subscriptions!',
 'Create a meal plan for the week!',
 'Calculate my carbon footprint based on my daily activities!',
 'Help me set reminders for my favorite TV shows!',
 'Assist me in managing my study schedule!',
 'Help me calculate the ROI for my investments!',
 'Assist me in tracking my water intake!',
 'Help me find the best time to post on social media based on my followers!',
 'Assist me in calculating my monthly budget!',
 'Calculate my coffee consumption']

In [ ]:
dataset["train"][0]["output"]

In [ ]:
def generate_prompt(data_point):
    """Generate input text based on a prompt, task instruction, (context info), and answer.

    :param data_point: dict: Data point
    :return: dict: Data point with the added "prompt" field
    """
    prefix_text = 'based on given instruction create a solution\n\n'
    prompt_text = f"""<start_of_turn>user {prefix_text} here are the instruction {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model {data_point["output"]} <end_of_turn>"""
    data_point["prompt"] = prompt_text
    return data_point

# Add the "prompt" column to the dataset
dataset = dataset['train'].map(generate_prompt)

# Print the updated dataset
dataset

In [ ]:
print(dataset[0]["prompt"])

In [ ]:
dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

In [ ]:
dataset

In [ ]:
dataset = dataset.train_test_split(test_size=0.1)
train_data = dataset["train"]
test_data = dataset["test"]

In [ ]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [ ]:
modules = find_all_linear_names(model)
print(modules)

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

In [ ]:
#new code using SFTTrainer
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,  # Adjust batch size based on GPU memory
        gradient_accumulation_steps=5,
        warmup_steps=100,  # Adjust warm-up steps
        num_train_epochs=1,  # Adjust the number of training epochs
        max_steps=500,  # Adjust the maximum number of steps
        learning_rate=1e-4,  # Adjust the learning rate
        logging_steps=10,  # Adjust how often training information is logged
        output_dir="python_model",  # Change the output directory
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [ ]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
new_model = "gemma-finetuned-python_code"
trainer.model.save_pretrained(new_model)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model_new",safe_serialization=True)
tokenizer.save_pretrained("merged_model_new_tokenizer")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

merged_model = AutoModelForCausalLM.from_pretrained("/kaggle/working/merged_model_new")
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/merged_model_new_tokenizer")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
query = input('enter a query:')
prompt_template = f"""
<start_of_turn>user based on given instruction create a solution\n\nhere are the instruction {query}
<end_of_turn>\n<start_of_turn>model
"""
prompt = prompt_template
encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True).input_ids

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
merged_model.to(device)
inputs = encodeds.to(device)


# Increase max_new_tokens if needed
generated_ids = merged_model.generate(inputs, max_new_tokens=1000, do_sample=False, pad_token_id=tokenizer.eos_token_id)
ans = ''
for i in tokenizer.decode(generated_ids[0], skip_special_tokens=True).split('<end_of_turn>')[:2]:
    ans += i

# Extract only the model's answer
model_answer = ans.split("model")[1].strip()
print(model_answer)

enter a query: Generate a code to implement the quicksort algorithm in Python


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Python code to implement the quicksort algorithm:

```python
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[len(arr) // 2]
        less = [x for x in arr if x < pivot]
        equal = [x for x in arr if x == pivot]
        greater = [x for x in arr if x > pivot]
        return quicksort(less) + equal + quicksort(greater)
```
This code uses the divide and conquer approach to sort an array. It first divides the array into two halves, then recursively sorts the two halves and finally merges them together.
```


In [37]:
print(dataset["train"][49007]["output"])

"""
Implement quicksort algorithm
"""
def partition(arr,low,high): 
    i = ( low-1 )          
    pivot = arr[high]     
    for j in range(low , high): 
        if arr[j] <= pivot:             
            i = i+1 
            arr[i],arr[j] = arr[j],arr[i] 
    arr[i+1],arr[high] = arr[high],arr[i+1] 
    return ( i+1 ) 
  
def quick_sort(arr,low,high): 
    if low < high: 
        pi = partition(arr,low,high) 
        quick_sort(arr, low, pi-1) 
        quick_sort(arr, pi+1, high) 

if __name__ == '__main__':
    arr = [9,4,2,7,3,8,5,6]
    n = len(arr) 
    quick_sort(arr,0,n-1)
    print("The sorted array is:")
    for i in arr: 
        print (i),


In [19]:
dataset["train"][49000:49010]["instruction"]

['Consider a way to detect malicious HTTP requests using Python',
 'Create a Python program to print out the lyrics of the song "Happy Birthday"',
 'Create a web-scraping program to extract data from twitter',
 'Create a Python program to convert a list of values from Celsius to Fahrenheit [0, 10, 20]',
 'Create an output array from the given input so that each element in the output is the product of all the other element in the same row [\n [10, 2, 3],\n [4, 5, 6],\n [7, 8, 9]\n]',
 'Generate a list comprehension in Python that prints the cubes of the first 5 numbers',
 'Write a python script that computes the sum of all the odd numbers between 1 and 1000',
 'Generate a code to implement the quicksort algorithm in Python',
 'Write a python program to transform input set of numbers into a list of tuples 1, 2, 3, 4, 5',
 'Create a Python program to scrape and store the HTML content of a given website']